In [224]:
#import libraries

from bs4 import BeautifulSoup
import requests
import time
import datetime

import smtplib
import re

import datetime

today = datetime.date.today()
print(today)

2024-08-13


In [225]:
#Connect to Website

URL = 'https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&crid=26XGF0ETVPQUI&qid=1723367023&sprefix=sun%2Caps%2C833&ref=sr_st_exact-aware-popularity-rank&ds=v1%3A3pDT%2BAG86jcKv9fBxlUHlsFORcPucnxPmCLjzaZBAa4'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"}

page = requests.get(URL, headers=headers)

soup = BeautifulSoup(page.content, "html.parser")
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

In [226]:
# functions

def get_title(soup):
    try:
        product_title = soup.find("span", attrs={"id":'productTitle'})
        title_value = product_title.text
        title_string = title_value.strip()
    
    except AttributeError:
        title_string = ""
    
    return title_string
    

def get_price(soup):
    price = soup.find(class_='a-offscreen')
    if price:
        price_text = price.text.strip()
        try:
            price = float(price_text[2:])
            return price
        except ValueError:
            return "N/A"
    else:
        return "N/A"

def get_rating(soup):
    rating = soup.find(class_='a-icon-alt')
    if rating:
        ratings = rating.text.split(' out of 5 stars')
        ratings = ratings[0]
        rat = ratings.strip()
        return rat
    else:
        return "N/A"
    
def get_reviews(soup):
    reviews_element = soup.find("span", attrs={'id':'acrCustomerReviewText'})
    if reviews_element:
        reviews = reviews_element.text.split(' ratings')
        reviews = reviews[0]
        rev = reviews.strip()
        return rev
    else:
        return "N/A"

def get_sales(soup):
    sales = soup.find(class_='a-size-small social-proofing-faceout-title-text')
    if sales:
        return sales.text
    else:
        return "N/A"

In [221]:
# Define the number of pages you want to scrape
num_pages = 7
base_url = 'https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&crid=26XGF0ETVPQUI&qid=1723367023&sprefix=sun%2Caps%2C833&ref=sr_st_exact-aware-popularity-rank&ds=v1%3A3pDT%2BAG86jcKv9fBxlUHlsFORcPucnxPmCLjzaZBAa4'

# Loop through each page

d = {'title': [], 'price': [], 'rating': [], 'reviews': [], 'monthly_sales': [], 'URL': []}

for page in range(1, num_pages +1):
    url = f'{base_url}{page}'
    response = requests.get(url, headers=headers)
    new_page_soup = BeautifulSoup(response.content, "html.parser")

    new_pages = new_page_soup.find_all('a', attrs={'class': 'a-link-normal s-no-outline'})

    for link in links:
        product_url = "https://www.amazon.sg" + link.get('href')
        new_product = requests.get(product_url, headers=headers)
        product_soup = BeautifulSoup(new_product.content, "html.parser")

        d['title'].append(get_title(product_soup))
        d['price'].append(get_price(product_soup))
        d['rating'].append(get_rating(product_soup))
        d['reviews'].append(get_reviews(product_soup))
        d['monthly_sales'].append(get_sales(product_soup))
        d['URL'].append(product_url)

amazon_df = pd.DataFrame.from_dict(d)
print(amazon_df)

                                                 title  price rating reviews  \
0       ANTHELIOS UVMUNE 400 fluide tinté SFP50+ 50 ml    N/A    4.5   2,644   
1    Neutrogena Purescreen+ Mineral Beach Defense O...  17.33    4.3      78   
2    Numbuzin No.1 Clear Filter Sun Essence Spf50+ ...    N/A    4.3     114   
3    All-Natural, Benzene Free, Non Nano Zinc Oxide...  30.36    4.2     522   
4    Eucerin Sun Sensitive Mineral Baby Sunscreen S...  24.94    4.5     429   
..                                                 ...    ...    ...     ...   
331  Evereden Sheer Botanical Face Sunscreen SPF 30...  47.13    4.3     116   
332  NEOVA DNA Damage Control Silc Sheer SPF 40, 2....    N/A    4.7     842   
333                                                  0    N/A    4.4      82   
334                                                       N/A    N/A     N/A   
335                                                       N/A    N/A     N/A   

    monthly_sales                      

In [228]:
import csv

with open('AmazonWebScraper_SunscreenSingapore_20240813.csv', 'w', newline='', encoding='UTF8') as f:
    amazon_df.to_csv('AmazonWebScraper_SunscreenSingapore_20240813.csv', index=False)

In [205]:
#Fetching Pages - OLD
pages = soup.find_all('a', class_='s-pagination-item s-pagination-button')

pages_list = ['https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&crid=26XGF0ETVPQUI&qid=1723484528&sprefix=sun%2Caps%2C833&ref=sr_pg_1']

for page in pages:
    page_url = 'https://www.amazon.sg' + page.get('href')
    pages_list.append(page_url)
    new_page = requests.get(page_url, headers=headers)
    new_page_soup = BeautifulSoup(new_page.content, "html.parser")
    
# Extract the URLs from the current page
    new_pages = new_page_soup.find_all('a', class_='s-pagination-item s-pagination-button')

    # Iterate through the new pages and add them to the list if they're not already in it
    for new_page in new_pages:
        new_url = 'https://www.amazon.sg' + new_page.get('href')
        if new_url not in pages_list:
            pages_list.append(new_url)
 
    #print(pages_list)

# Create a dictionary to store unique URLs based on last 4 characters
unique_urls = {}
for url in pages_list:
    last_four = url[-4:]
    if last_four not in unique_urls:
        unique_urls[last_four] = url
        
# Convert the dictionary back to a list
page_link_list = list(unique_urls.values())

print(page_link_list)

['https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&crid=26XGF0ETVPQUI&qid=1723484528&sprefix=sun%2Caps%2C833&ref=sr_pg_1', 'https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&page=2&crid=26XGF0ETVPQUI&qid=1723525138&sprefix=sun%2Caps%2C833&ref=sr_pg_2', 'https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&page=3&crid=26XGF0ETVPQUI&qid=1723525140&sprefix=sun%2Caps%2C833&ref=sr_pg_3', 'https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&page=4&crid=26XGF0ETVPQUI&qid=1723525141&sprefix=sun%2Caps%2C833&ref=sr_pg_4']


In [206]:
# Create a dictionary to store product details - OLD
d = {'title': [], 'price': [], 'rating': [], 'reviews': [], 'monthly_sales': [], 'URL': []}

# Loop through the unique URLs and extract the product details
for unique_url in page_link_list:
    unique_page = requests.get(unique_url, headers=headers)
    product_soup = BeautifulSoup(unique_page.content, "html.parser")
    links = product_soup.find_all('a', attrs={'class': 'a-link-normal s-no-outline'})

    for link in links:
        product_url = "https://www.amazon.sg" + link.get('href')
        new_product = requests.get(product_url, headers=headers)
        product_soup = BeautifulSoup(new_product.content, "html.parser")

        d['title'].append(get_title(product_soup))
        d['price'].append(get_price(product_soup))
        d['rating'].append(get_rating(product_soup))
        d['reviews'].append(get_reviews(product_soup))
        d['monthly_sales'].append(get_sales(product_soup))
        d['URL'].append(product_url)

amazon_df = pd.DataFrame.from_dict(d)
print(amazon_df)

                                                 title  price rating reviews  \
0    CeraVe 100% Mineral Sunscreen SPF 50 | Face Su...    N/A    4.3  76,598   
1    Aveeno Kids Continuous Protection Zinc Oxide M...   18.4    4.6   1,393   
2    Aveeno Protect + Hydrate Moisturizing Body Sun...  18.67    4.5   9,275   
3    BLUE LIZARD Sensitive Mineral Sunscreen with Z...  21.27    4.7  15,963   
4    CeraVe Anti Aging Face Cream with SPF 30 Sunsc...    N/A    4.3   8,985   
..                                                 ...    ...    ...     ...   
187  Evereden Sheer Botanical Face Sunscreen SPF 30...  47.13    4.3     116   
188  NEOVA DNA Damage Control Silc Sheer SPF 40, 2....    N/A    4.7     841   
189                                                  0    N/A    4.4      82   
190  Cancer Council Face Day Wear Moisturiser Invis...    N/A    5.0       6   
191  Kiehl's Ultra Facial Cream with SPF 30, Lightw...    N/A    4.6     346   

                  monthly_sales  \
0   

In [ ]:
#Appending data to the csv

with open('AmazonWebScraper_SunscreenSingapore.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)